## 비정형 데이터 유효성 검증

#### 파싱한 함수 위에 배치 나중에는 py에 넣어서

In [9]:
# OCR 파트
import json
import time
import requests
from pathlib import Path
import re
import fitz  # PyMuPDF

import os
from dotenv import load_dotenv

# 파일별 파싱 함수
from pathlib import Path
import pandas as pd
from docx import Document
import pdfplumber

load_dotenv()

INVOKE_URL = os.getenv("CLOVA_INVOKE_URL")
SECRET_KEY = os.getenv("CLOVA_OCR_SECRET")

# 파일(이미지/페이지)을 bytes 형태로 Clova OCR API에 보내서 OCR 결과 JSON을 받아옴
def _call_clova_from_bytes(
    filename: str,
    file_bytes: bytes,
    fmt: str,
    lang: str = "ko",
    enable_table: bool = True,
    timeout: int = 180,
) -> dict:
    headers = {"X-OCR-SECRET": SECRET_KEY}
    message = {
        "version": "V1",
        "requestId": f"clova-{Path(filename).stem}-{int(time.time()*1000)}",
        "timestamp": int(time.time() * 1000),
        "lang": lang,
        "images": [{"format": fmt, "name": Path(filename).stem}],
        "enableTableDetection": bool(enable_table),
    }
    files = {
        "file": (filename, file_bytes, "application/octet-stream"),
        "message": (None, json.dumps(message), "application/json"),
    }
    r = requests.post(INVOKE_URL, headers=headers, files=files, timeout=timeout)
    r.raise_for_status()
    return r.json()

# PDF를 페이지별로 렌더링해서 PNG 이미지 바이트로 변환
def _pdf_to_png_bytes_list(pdf_path: str, dpi: int = 200) -> list[tuple[str, bytes]]:
    p = Path(pdf_path)
    doc = fitz.open(str(p))
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)

    out: list[tuple[str, bytes]] = []
    for i in range(len(doc)):
        pix = doc[i].get_pixmap(matrix=mat, alpha=False)
        out.append((f"{p.stem}_p{i+1}.png", pix.tobytes("png")))
    doc.close()
    return out

# Clova가 준 테이블 셀 1개에서 텍스트를 뽑는데, 셀 안에 여러 줄이면 줄바꿈(\n)을 유지해서 합침
def _cell_text_keep_lines(cell: dict) -> str:
    lines: list[str] = []

    if isinstance(cell.get("cellTextLines"), list) and cell["cellTextLines"]:
        for ln in cell["cellTextLines"]:
            if isinstance(ln.get("cellWords"), list) and ln["cellWords"]:
                s = " ".join(
                    [w.get("inferText", "") for w in ln["cellWords"] if w.get("inferText")]
                )
            else:
                s = ln.get("inferText", "")
            s = re.sub(r"\s+", " ", str(s)).strip()
            if s:
                lines.append(s)

    elif isinstance(cell.get("cellWords"), list) and cell["cellWords"]:
        s = " ".join([w.get("inferText", "") for w in cell["cellWords"] if w.get("inferText")])
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    else:
        s = (cell.get("inferText") or cell.get("text") or "")
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    return "\n".join(lines).strip()

# Clova OCR 결과 중에서 페이지의 일반 텍스트 영역(fields)을 전부 이어 붙여 페이지 전체 텍스트로 만듬
def _page_text_from_fields(result_json: dict) -> str:
    img0 = (result_json.get("images") or [{}])[0]
    fields = img0.get("fields") or []
    return " ".join([f.get("inferText", "") for f in fields if f.get("inferText")]).strip()

# 테이블에서 rowSpan / colSpan(병합셀) 정보를 보고, 병합된 셀의 텍스트를 병합 영역 전체 좌표에 복제해서 셀 리스트 만듬
def _merge_cells_with_rowspan(cells: list[dict]) -> list[dict]:
    has_span = any(("rowSpan" in c) or ("colSpan" in c) for c in cells)
    if not has_span:
        return cells

    expanded_cells: list[dict] = []
    for cell in cells:
        row_idx = cell.get("rowIndex", 0)
        col_idx = cell.get("columnIndex", 0)
        row_span = cell.get("rowSpan", 1) or 1
        col_span = cell.get("colSpan", 1) or 1
        text = _cell_text_keep_lines(cell)

        for r in range(row_idx, row_idx + row_span):
            for c in range(col_idx, col_idx + col_span):
                expanded_cells.append(
                    {
                        "rowIndex": r,
                        "columnIndex": c,
                        "text": text,
                        "isMerged": (r != row_idx or c != col_idx),
                    }
                )

    return expanded_cells

# Clova OCR 결과에서 tables를 꺼내서 2차원 배열(grid) 형태로 정리
def _tables_to_schema(result_json: dict) -> dict:
    img0 = (result_json.get("images") or [{}])[0]
    tables = img0.get("tables") or []

    out: dict = {}
    for ti, t in enumerate(tables, start=1):
        cells = t.get("cells") or []
        if not cells:
            out[f"table{ti}"] = {"rows": [], "rowCount": 0, "colCount": 0}
            continue

        has_rc = any("rowIndex" in c for c in cells) and any("columnIndex" in c for c in cells)
        if not has_rc:
            texts = [_cell_text_keep_lines(c) for c in cells]
            texts = [x for x in texts if x]
            rows = [[x] for x in texts]
            out[f"table{ti}"] = {"rows": rows, "rowCount": len(rows), "colCount": 1 if rows else 0}
            continue

        expanded_cells = _merge_cells_with_rowspan(cells)

        max_r = max(c.get("rowIndex", 0) for c in expanded_cells)
        max_c = max(c.get("columnIndex", 0) for c in expanded_cells)
        grid = [[""] * (max_c + 1) for _ in range(max_r + 1)]

        for c in expanded_cells:
            r = c.get("rowIndex", 0)
            col = c.get("columnIndex", 0)
            text = c.get("text", "")
            if not grid[r][col]:
                grid[r][col] = text

        out[f"table{ti}"] = {
            "rows": grid,
            "rowCount": len(grid),
            "colCount": len(grid[0]) if grid else 0,
        }

    return out

# 전체 파이프라인 엔트리 함수
def clovaOCR(file_path: str, lang: str = "ko", dpi: int = 200, enable_table: bool = True) -> dict:
    p = Path(file_path)
    if not p.exists():
        raise FileNotFoundError(str(p.resolve()))

    pages_out: dict = {}

    if p.suffix.lower() == ".pdf":
        page_pngs = _pdf_to_png_bytes_list(str(p), dpi=dpi)

        for i, (fname, png_bytes) in enumerate(page_pngs, start=1):
            print(f"[{i}/{len(page_pngs)}] OCR...") # OCR 되는지 확인 용 디버깅
            rj = _call_clova_from_bytes(
                fname, png_bytes, fmt="png", lang=lang, enable_table=enable_table
            )
            tables = _tables_to_schema(rj)

            pages_out[f"page{i}"] = {
                "text": _page_text_from_fields(rj),
                "tableCount": len(tables),
                "tables": tables,
            }

        return {"pageCount": len(page_pngs), "pages": pages_out}

    with open(p, "rb") as f:
        b = f.read()

    fmt = p.suffix.lower().lstrip(".")
    if fmt == "jpeg":
        fmt = "jpg"

    rj = _call_clova_from_bytes(p.name, b, fmt=fmt, lang=lang, enable_table=enable_table)
    tables = _tables_to_schema(rj)

    pages_out["page1"] = {
        "text": _page_text_from_fields(rj),
        "tableCount": len(tables),
        "tables": tables,
    }
    return {"pageCount": 1, "pages": pages_out}

# 표처리 헬퍼
def clean_table(tbl):   
    return [
        [c for c in ("" if c is None else str(c).strip() for c in row) if c]
        for row in tbl
        if any("" if c is None else str(c).strip() for c in row)
    ]


CSV_EXT = {"csv"}
EXCEL_EXT = {"xls", "xlsx"}
IMAGE_EXT = {"jpg", "png"}
OTHER_EXT = {"pdf", "docx"}

def handle_csv(path: str, slotName: str):
    for enc in ["euc-kr", "cp949", "utf-8"]:
        try:
            df = pd.read_csv(path, encoding=enc)
            return {"slotName": slotName, "kind": "CSV", "ext": "csv", "dataframe": df}
        except UnicodeDecodeError:
            continue
    raise ValueError("인코딩 실패: cp949, euc-kr, utf-8 encoding을 지원합니다")

def handle_excel(path: str, ext: str, slotName: str):
    df = pd.read_excel(path)
    return {"slotName": slotName, "kind": "EXCEL", "ext": ext, "dataframe": df}

def handle_image(path: str, ext: str, slotName: str):
    ocr_output = clovaOCR(path)
    return {"slotName": slotName, "kind": "IMAGE", "ext": ext, "content": ocr_output}

def clean_table(tbl):
    return [
        [c for c in ("" if c is None else str(c).strip() for c in row) if c]
        for row in tbl
        if any("" if c is None else str(c).strip() for c in row)
    ]

def handle_pdf(path: str, ext: str, slotName: str):
    PDF_PAGE_MIN_CHARS = 30
    PDF_PASS_RATIO = 0.7

    with pdfplumber.open(path) as pdf:
        total_pages = len(pdf.pages)
        valid_page_count = 0
        pages_data = {}

        for i, page in enumerate(pdf.pages):
            t = page.extract_text() or ""
            clean_t = " ".join(t.split()).strip()
            if len(clean_t) >= PDF_PAGE_MIN_CHARS: valid_page_count += 1
            raw = page.extract_tables() or []
            tables = {f"table{ti+1}": (lambda rows: {"rows": rows, "rowCount": len(rows), "colCount": max((len(r) for r in rows), default=0)})(clean_table(tbl)) for ti, tbl in enumerate(raw)}
            pages_data[f"page{i+1}"] = {"text": clean_t, "tableCount": len(raw), "tables": tables}

        pass_ratio = valid_page_count / max(total_pages, 1)

    if pass_ratio >= PDF_PASS_RATIO: return {"slotName": slotName, "kind":"PDF","ext":ext,"mode":"text","pass_ratio":pass_ratio,"content":{"pageCount":total_pages,"pages":pages_data}}
    else:
        ocr_output = clovaOCR(path)
        return {"slotName": slotName, "kind": "PDF", "ext": ext, "mode": "ocr", "content": ocr_output}

def handle_docx(path: str, ext: str, slotName: str):
    doc = Document(path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    tables = {}
    for i, table in enumerate(doc.tables, start=1):
        rows = [[cell.text for cell in row.cells] for row in table.rows]
        tables[f"table{i}"] = {
            "rows": rows,
            "rowCount": len(rows),
            "colCount": max((len(r) for r in rows), default=0),
        }
    pages = {
        "page1": {
            "text": "\n".join(paragraphs),
            "tableCount": len(doc.tables),
            "tables": tables,
        }
    }
    return {"slotName": slotName, "kind": "DOCX", "ext": ext, "content": {"pageCount": 1, "pages": pages}}

# 파일 형식별 조건부 함수구동
def handle_file(path: str, slotName: str):
    ext = Path(path).suffix.lower().lstrip(".")
    if ext in CSV_EXT: return handle_csv(path, slotName)
    if ext in EXCEL_EXT: return handle_excel(path, ext, slotName)
    if ext in IMAGE_EXT: return handle_image(path, ext, slotName)
    if ext == "pdf": return handle_pdf(path, ext, slotName)
    if ext == "docx": return handle_docx(path, ext, slotName)
    raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

## 0) 목표 출력 포맷(네 요구 반영)<br>

**PASS (비정형: payload 없어도 됨)**

- status, file_name, validated_fields, processed_at 필수
- payload는 있으면 추가, 비정형은 보통 생략

**FAIL**

- status, error, file_name, processed_at 필수
- error.location은 비정형 기준으로 page/snippet 형태로 넣어주면 좋음

#### 1) (셀1) 슬롯 규칙 정의 (4~15번: slotName별 키워드/검증필드)

In [87]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple
from datetime import datetime, timezone
import re

# 공통 기본값(유효성 검증 기준)
DEFAULT_REQUIRED_MIN_HITS = 1  # 키워드 최소 매칭 개수(3개 중 1개라도 있으면 통과)
DEFAULT_MIN_TEXT_LEN = 80      # 텍스트 최소 길이(너무 짧으면 OCR/파싱 실패로 간주)

# 공통 검증 체크 항목
COMMON_VALIDATED_FIELDS = (
    "pageCount_valid",       # 페이지 수가 0이 아닌지
    "text_present",          # 텍스트가 비어있지 않은지
    "min_text_len_ok",       # 텍스트 길이가 최소 기준 이상인지
    "keyword_min_hits_ok",   # 키워드 규칙(최소 매칭 개수)을 만족하는지
)

@dataclass
class SlotSpec:
    slot_name: str
    required_keywords: List[str]
    required_min_hits: int = DEFAULT_REQUIRED_MIN_HITS
    min_text_len: int = DEFAULT_MIN_TEXT_LEN

    # 공통값 자동 주입 (슬롯에서 매번 validated_fields 넣을 필요 없음)
    validated_fields: List[str] = field(default_factory=lambda: list(COMMON_VALIDATED_FIELDS))


SLOT_SPECS: Dict[str, SlotSpec] = {
    # 4 환경경영 인증 - ISO 14001 인증서
    "iso_14001_certificate": SlotSpec(
        slot_name="iso_14001_certificate",
        required_keywords=["ISO 14001", "Effective Date", "Expiration Date"],
    ),

    # 5 대기오염 관리 - 대기 자가측정 기록부
    "air_self_measurement_log": SlotSpec(
        slot_name="air_self_measurement_log",
        required_keywords=["대기분야 측정기록부", "배출가스", "NOx"],
    ),

    # 6 유해물질 관리 - MSDS
    "msds": SlotSpec(
        slot_name="msds",
        required_keywords=["물질안전보건자료", "Material Safety Data Sheet", "CAS"],
    ),

    # 7 임직원 현황 - 인사현황표/임금대장
    "employee_status": SlotSpec(
        slot_name="employee_status",
        required_keywords=["임원 및 직원 등에 관한 사항", "임원 및 직원 등의 현황", "직원 등 현황"],
    ),

    # 8 안전 교육 이수 - 안전보건교육 실시 결과 보고서
    "safety_training_report": SlotSpec(
        slot_name="safety_training_report",
        required_keywords=["안전보건 교육일지", "교육 대상자 수", "교육 실시자 수"],
    ),

    # 9 산업재해 발생 - 산업재해 기록부/무재해 증명
    "industrial_accident_record": SlotSpec(
        slot_name="industrial_accident_record",
        required_keywords=["산업재해조사표", "산재관리번호", "재해발생원인"],
    ),

    # 10 안전보건 인증 - ISO 45001 인증서
    "iso_45001_certificate": SlotSpec(
        slot_name="iso_45001_certificate",
        required_keywords=["ISO 45001", "Effective Date", "Expiration Date"],
    ),

    # 11 근로권 준수 - 취업규칙/근로계약서 양식 (docx)
    "labor_rules_or_contract": SlotSpec(
        slot_name="labor_rules_or_contract",
        required_keywords=["근로계약", "근로 및 휴식시간", "임금 및 휴무"],
    ),

    # 12 아동/강제노동 금지 - 인권 선언문/윤리강령 (docx)
    "no_child_forced_labor_policy": SlotSpec(
        slot_name="no_child_forced_labor_policy",
        required_keywords=["아동노동", "강제노동", "금지"],
    ),

    # 13 윤리강령
    "code_of_ethics": SlotSpec(
        slot_name="code_of_ethics",
        required_keywords=["윤리강령", "부패", "신고"],
    ),

    # 14 이사회 운영 현황 (사업보고서 내 이사회 파트)
    "board_operation": SlotSpec(
        slot_name="board_operation",
        required_keywords=["이사회", "사외이사", "주요 의결사항"],
    ),

    # 15 이사의 보수 (사업보고서 내 임원의 보수 파트)
    "director_compensation": SlotSpec(
        slot_name="director_compensation",
        required_keywords=["임원의 보수", "보수총액", "산정기준 및 방법"],
    ),
}

#### 2) (셀2) 공통 유틸: 텍스트 정리 / 키워드 검사 / 스니펫 만들기

In [88]:
# 현재 시간을 UTC 기준 ISO 문자열(예: 2026-01-20T01:10:54Z)로 반환
def now_iso_utc() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# OCR/파싱 텍스트 정리: NBSP 제거 + 공백/줄바꿈을 한 칸으로 통일 + 양끝 공백 제거
def normalize_text(s: str) -> str:
    if not s:
        return ""
    s = s.replace("\u00a0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# 키워드 규칙 검사: keywords 중 최소 min_hits개 이상 발견되면 OK (hits/missing도 같이 반환)
def keyword_min_hits(text: str, keywords: List[str], min_hits: int = 1) -> Tuple[bool, List[str], List[str]]:
    t = (text or "").lower()
    hits = [kw for kw in keywords if kw.lower() in t]
    missing = [kw for kw in keywords if kw.lower() not in t]
    ok = len(hits) >= (min_hits or 1)
    return ok, hits, missing

# 미리보기용 스니펫 생성: 텍스트를 정리한 뒤 max_len까지만 잘라서 반환
def make_snippet(text: str, max_len: int = 180) -> str:
    t = normalize_text(text)
    if len(t) <= max_len:
        return t
    return t[:max_len] + "..."

#### 3) (셀3) 비정형 OCR JSON → 문서 요약(meta) 뽑기

**너가 준 OCR/파싱 JSON 구조에 맞춰서:**

- pageCount
- page별 text 길이
- tableCount
- 전체 text 합치기

In [89]:
# OCR/파싱 결과(JSON)에서 페이지/텍스트/테이블 정보를 요약(meta)으로 변환(검증용 full_text, page별 길이/스니펫 생성)
def summarize_ocr_content(parsed: Dict[str, Any]) -> Dict[str, Any]:
    slot_name = parsed.get("slotName", "") or ""
    kind = parsed.get("kind", "") or ""
    ext = parsed.get("ext", "") or ""
    mode = parsed.get("mode", "") or ""

    content = parsed.get("content", {}) or {}
    page_count = int(content.get("pageCount", 0) or 0)
    pages = (content.get("pages", {}) or {})

    page_summaries: List[Dict[str, Any]] = []
    full_text_parts: List[str] = []
    total_table_count = 0

    # page1, page2, ... 순서대로 정렬
    def page_sort_key(k: str) -> int:
        m = re.search(r"(\d+)$", k)
        return int(m.group(1)) if m else 999999

    for page_key in sorted(pages.keys(), key=page_sort_key):
        page_obj = pages.get(page_key, {}) or {}

        page_text = page_obj.get("text", "") or ""
        page_text_norm = normalize_text(page_text)

        if page_text_norm:
            full_text_parts.append(page_text_norm)

        table_count = int(page_obj.get("tableCount", 0) or 0)
        total_table_count += table_count

        page_summaries.append({
            "page": page_key,
            "text_len": len(page_text_norm),
            "tableCount": table_count,
            "snippet": make_snippet(page_text_norm),
        })

    # 전체 텍스트 합치기 + 한번 더 normalize
    full_text = normalize_text(" ".join(full_text_parts))
    total_text_len = len(full_text)

    # 디버깅 편의: 텍스트가 가장 긴 페이지(대표 페이지) 하나 뽑기
    top_page = None
    if page_summaries:
        top_page = max(page_summaries, key=lambda x: x.get("text_len", 0))

    return {
        "slotName": slot_name,
        "kind": kind,
        "ext": ext,
        "mode": mode,
        "pageCount": page_count,
        "total_text_len": total_text_len,
        "total_table_count": total_table_count,
        "has_table": total_table_count > 0,   # (선택) 나중에 쓰기 편함
        "pages": page_summaries,
        "top_page": top_page,                # (중요) 실패 시 근거로 쓰기 좋음
        "full_text": full_text,
    }


#### 4) (셀4) 핵심: 슬롯별 비정형 유효성 검증 함수

**검증 규칙(너가 말한 최소 기준 반영):**

- pageCount 존재/정상
- text 비어있지 않음 (전체 텍스트 길이, 첫 페이지 길이)
- 필수 키워드 포함 (slotName별)
- PASS/FAIL 결과 포맷 맞추기
    - 비정형은 payload 생략 가능
    - validated_fields, processed_at 항상 포함

In [90]:
# PASS 결과 포맷 생성(비정형은 payload 생략 가능, validated_fields/processed_at은 항상 포함)
def make_pass(file_name: str, validated_fields: List[str], payload: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    res = {
        "status": "PASS",
        "file_name": file_name,
        "validated_fields": validated_fields,
        "processed_at": now_iso_utc()
    }
    if payload:
        res["payload"] = payload
    return res


# FAIL 결과 포맷 생성(에러 코드/메시지 + 근거 위치(location) 포함)
def make_fail(
    file_name: str,
    code: str,
    message: str,
    location: Optional[str] = None,
    issues: Optional[List[Dict[str, Any]]] = None
) -> Dict[str, Any]:
    err = {"code": code, "message": message}
    if location:
        err["location"] = location
    if issues:
        err["issues"] = issues  # 여러 문제를 한 번에 전달

    return {
        "status": "FAIL",
        "error": err,
        "file_name": file_name,
        "processed_at": now_iso_utc()
    }

# 슬롯별 스펙(SLOT_SPECS)에 따라 OCR/파싱 결과를 최소 기준(page/text/키워드)로 PASS/FAIL 판정
def validate_unstructured_slot(parsed: Dict[str, Any]) -> Dict[str, Any]:
    summary = summarize_ocr_content(parsed)
    slot_name = summary.get("slotName", "") or "unknown_slot"

    # 1) 스펙 존재 여부
    spec = SLOT_SPECS.get(slot_name)
    if spec is None:
        return make_fail(
            file_name=slot_name,
            code="UNKNOWN_SLOT",
            message=f"등록되지 않은 slotName 입니다: {slot_name}",
            location="slotName"
        )

    # 대표 페이지(근거용): top_page 있으면 그걸 사용, 없으면 첫 페이지
    ref_page = summary.get("top_page") or (summary["pages"][0] if summary.get("pages") else None)
    ref_loc = None
    if ref_page:
        ref_loc = f'{ref_page["page"]}:snippet="{ref_page.get("snippet","")}"'

    # 여러 문제를 한 번에 모으기
    problems: List[str] = []

    # 2) 기본 구조 검사
    if summary.get("pageCount", 0) <= 0:
        problems.append("EMPTY_PAGECOUNT: pageCount가 0이거나 누락되어 문서를 처리할 수 없습니다.")

    if not summary.get("pages"):
        problems.append("EMPTY_PAGES: pages 정보가 없어 문서를 처리할 수 없습니다.")

    # 3) 텍스트 존재성 검사
    total_len = summary.get("total_text_len", 0) or 0
    if total_len == 0:
        problems.append("EMPTY_TEXT: 텍스트가 비어있습니다(OCR/파싱 실패 가능).")
    elif total_len < spec.min_text_len:
        problems.append(f"TEXT_TOO_SHORT: OCR 텍스트가 너무 짧습니다(총 {total_len}자 < 최소 {spec.min_text_len}자).")

    # 4) 필수 키워드 검사
    ok_kw, hits, missing = keyword_min_hits(
        text=summary.get("full_text", ""),
        keywords=spec.required_keywords,
        min_hits=spec.required_min_hits
    )
    if not ok_kw:
        problems.append(f"필수 키워드 매칭 실패(최소 {spec.required_min_hits}개 필요): 누락={missing}")

    # 문제 있으면 FAIL 1번만 반환(메시지에 전부 포함)
    if problems:
        return make_fail(
            file_name=slot_name,
            code="VALIDATION_FAILED",
            message="; ".join(problems),
            location=ref_loc or "content.pages"
        )

    # PASS (비정형은 payload 생략)
    return make_pass(
        file_name=slot_name,
        validated_fields=spec.validated_fields,
        payload=None
    )

In [91]:
path = "../data/samples/E/성광벤드_ISO_14001.pdf"
slot_name = "iso_14001_certificate"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'PASS',
 'file_name': 'iso_14001_certificate',
 'validated_fields': ['pageCount_valid',
  'text_present',
  'min_text_len_ok',
  'keyword_min_hits_ok'],
 'processed_at': '2026-01-20T04:47:28Z'}

In [92]:
path = "../data/samples/E/성광벤드_대기 측정기록부(환경분야 시험ㆍ검사 등에 관한 법률 시행규칙).pdf"
slot_name = "air_self_measurement_log"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'PASS',
 'file_name': 'air_self_measurement_log',
 'validated_fields': ['pageCount_valid',
  'text_present',
  'min_text_len_ok',
  'keyword_min_hits_ok'],
 'processed_at': '2026-01-20T04:47:31Z'}

In [93]:
path = "../data/samples/S/근로계약서_서약서_지적사항강화_샘플.docx"
slot_name = "labor_rules_or_contract"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'PASS',
 'file_name': 'labor_rules_or_contract',
 'validated_fields': ['pageCount_valid',
  'text_present',
  'min_text_len_ok',
  'keyword_min_hits_ok'],
 'processed_at': '2026-01-20T04:47:31Z'}

In [94]:
path = "../data/samples/S/윤리강령_샘플_아동강제노동포함.docx"
slot_name = "no_child_forced_labor_policy"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'PASS',
 'file_name': 'no_child_forced_labor_policy',
 'validated_fields': ['pageCount_valid',
  'text_present',
  'min_text_len_ok',
  'keyword_min_hits_ok'],
 'processed_at': '2026-01-20T04:47:31Z'}

file_name: 현재 slot_name이라서 demo에서는 그대로 이제 파이프라인 연결때 파일 이름으로 변경

In [95]:
path = "../data/samples/S/근로계약서_서약서_지적사항강화_샘플2.docx"
slot_name = "labor_rules_or_contract"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'FAIL',
 'error': {'code': 'VALIDATION_FAILED',
  'message': 'TEXT_TOO_SHORT: OCR 텍스트가 너무 짧습니다(총 58자 < 최소 80자).',
  'location': 'page1:snippet="근로계약서 및 서약서 본 근로계약은 회사와 근로자 간의 근로관계를 명확히 하기 위하여 체결되며, 근로자는"'},
 'file_name': 'labor_rules_or_contract',
 'processed_at': '2026-01-20T04:47:31Z'}

In [96]:
path = "../data/samples/S/윤리강령_샘플_아동강제노동포함2.docx"
slot_name = "no_child_forced_labor_policy"

parsed = handle_file(path, slot_name)
parsed

validation = validate_unstructured_slot(parsed)
validation

{'status': 'FAIL',
 'error': {'code': 'VALIDATION_FAILED',
  'message': "TEXT_TOO_SHORT: OCR 텍스트가 너무 짧습니다(총 79자 < 최소 80자).; 필수 키워드 매칭 실패(최소 1개 필요): 누락=['아동노동', '강제노동', '금지']",
  'location': 'page1:snippet="강령 (Code of Conduct) 제1조 목적 대표이사 서명: __________________________ 시행일: 2025-01-01"'},
 'file_name': 'no_child_forced_labor_policy',
 'processed_at': '2026-01-20T04:47:31Z'}